# Final Project - Advanced Deep Learning

### Data Preparation

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import zipfile
import requests

def download_file(url, local_filename):
    # Check if file already exists locally
    if os.path.exists(local_filename):
        print(f"{local_filename} already exists locally.")
        return

    # Download the file
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Ensure we raise an error for bad responses
        with open(local_filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    print(f"Downloaded {local_filename} from {url}.")


def unzip_file(zip_filepath, extract_to):
    # Check if the extracted directory already exists
    if os.path.isdir(extract_to):
        print(f"Directory {extract_to} already exists. Skipping extraction.")
        return

    # Unzip the file
    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_filepath} to {extract_to}.")



# this url is available for only a limited time.
dataset_url = "http://t12s-418866851410-ltu-adl.s3-website.eu-central-1.amazonaws.com/chest_xray.zip"
local_zip_filename = "/tmp/chest_xray.zip"
extract_to = "/tmp/ltu-chest-xray"

# Download the file if not already downloaded
download_file(dataset_url, local_zip_filename)

# Unzip the file if not already unzipped
unzip_file(local_zip_filename, extract_to)


# Define directories
train_dir = '/tmp/ltu-chest-xray/chest_xray/train/'
val_dir = '/tmp/ltu-chest-xray/chest_xray/val/'
test_dir = '/tmp/ltu-chest-xray/chest_xray/test/'

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


### Model Development

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)


### Training

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_data=val_generator,
    validation_steps=max(1, val_generator.samples // val_generator.batch_size),
    epochs=25
)

### Evaluation

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator, steps=max(1, test_generator.samples // test_generator.batch_size))
print(f'Test Accuracy: {test_acc}')

# Predict the labels for test set
Y_pred = model.predict(test_generator, steps=(test_generator.samples // test_generator.batch_size) + 1)
y_pred = np.round(Y_pred).astype(int)

# Ensuring y_pred matches the number of test samples
y_pred = y_pred[:test_generator.samples]

y_true = test_generator.classes

# Calculate F-score
from sklearn.metrics import f1_score, precision_score, recall_score

f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f'F1 Score: {f1}, Precision: {precision}, Recall: {recall}')


### Visualization

In [ ]:
# Visualization with TensorBoard
import datetime
from tensorflow.keras.callbacks import TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:

# Visualization with Weights & Biases
!pip install wandb
import wandb
from wandb.integration.keras import WandbCallback

wandb.init(project="pneumonia-detection")
model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_data=val_generator,
    validation_steps=max(1, val_generator.samples // val_generator.batch_size),
    epochs=25,
    callbacks=[WandbCallback()]
)